In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 27.2 MB/s  0:00:01 29.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 MB 34.1 MB/s  0:00:024.6 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 37.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 13.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.6/782.6 kB 21.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.9/36.9 MB 24.5 MB/s  0:00:014.1 MB/s eta 0:00:01:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20/20 [ultralytics]m 19/20 [ultralytics]n]-32]


In [ ]:
!pip install ultralytics

Cloning into 'YOLOv11-seg'...
remote: Repository not found.
fatal: repository 'https://github.com/yourrepo/YOLOv11-seg.git/' not found


In [ ]:
import yaml
from pathlib import Path

import torch

from ultralytics import YOLO

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Device:", device)

Device: cpu


In [14]:
data_yaml_path = "./dataset/data.yaml"

In [ ]:
with open(data_yaml_path) as f:
    data = yaml.safe_load(f)

print("Classes:", data['names'])
print("Train images:", len(list(Path(data['train']).rglob("*.jpg"))))
print("Val images:", len(list(Path(data['val']).rglob("*.jpg"))))

Classes: ['Kolesa', 'fire', 'insiderail', 'lep', 'outsiderail', 'people', 'perevod', 'rail', 'something', 'stolb', 'train', 'znak']
Train images: 0
Val images: 0


In [ ]:
model = YOLO("yolo11l-seg.pt")

In [ ]:
model.train(
    data=data_yaml_path,
    epochs=200,
    imgsz=1024,
    batch=8,
    lr0=0.0025,
    lrf=0.01,
    warmup_epochs=3,
    patience=30,
    weight_decay=0.0005,
    optimizer="AdamW",
    mosaic=1.0
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    degrees=10.0, translate=0.1, scale=0.5, shear=2.0,
    perspective=0.0005, flipud=0.5, fliplr=0.5,
    mixup=0.1, copy_paste=0.2,
    augment=True,
    cache=True,
    pretrained=True,
    device=0,
    workers=8,
    accumulate=2
)

Ultralytics 8.3.221 🚀 Python-3.10.18 torch-2.9.0 CPU (Apple M4 Max)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11_seg_experiment, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0

/opt/anaconda3/envs/anaconda-python-3-10/lib/python3.10/site-packages/ultralytics/data/augment.py:1248: RuntimeWarning: divide by zero encountered in matmul
  xy = xy @ M.T  # transform
/opt/anaconda3/envs/anaconda-python-3-10/lib/python3.10/site-packages/ultralytics/data/augment.py:1248: RuntimeWarning: overflow encountered in matmul
  xy = xy @ M.T  # transform
/opt/anaconda3/envs/anaconda-python-3-10/lib/python3.10/site-packages/ultralytics/data/augment.py:1248: RuntimeWarning: invalid value encountered in matmul
  xy = xy @ M.T  # transform


      1/100         0G      1.655      4.401      3.993      1.623        200        640: 100% ━━━━━━━━━━━━ 4/4 0.2it/s 16.8s3.8ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 0.4it/s 5.3s15.9s
                   all         38        302     0.0113      0.302     0.0348     0.0176     0.0068      0.186     0.0273     0.0131

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


/opt/anaconda3/envs/anaconda-python-3-10/lib/python3.10/site-packages/ultralytics/data/augment.py:1248: RuntimeWarning: divide by zero encountered in matmul
  xy = xy @ M.T  # transform
/opt/anaconda3/envs/anaconda-python-3-10/lib/python3.10/site-packages/ultralytics/data/augment.py:1248: RuntimeWarning: overflow encountered in matmul
  xy = xy @ M.T  # transform
/opt/anaconda3/envs/anaconda-python-3-10/lib/python3.10/site-packages/ultralytics/data/augment.py:1248: RuntimeWarning: invalid value encountered in matmul
  xy = xy @ M.T  # transform


      2/100         0G      1.599      4.325      3.959      1.623        144        640: 100% ━━━━━━━━━━━━ 4/4 0.2it/s 16.1s3.6ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 0.4it/s 5.3s16.0s
                   all         38        302     0.0138       0.31     0.0499     0.0251    0.00827      0.203     0.0368      0.019

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

/opt/anaconda3/envs/anaconda-python-3-10/lib/python3.10/site-packages/ultralytics/data/augment.py:1248: RuntimeWarning: divide by zero encountered in matmul
  xy = xy @ M.T  # transform
/opt/anaconda3/envs/anaconda-python-3-10/lib/python3.10/site-packages/ultralytics/data/augment.py:1248: RuntimeWarning: overflow encountered in matmul
  xy = xy @ M.T  # transform
/opt/anaconda3/envs/anaconda-python-3-10/lib/python3.10/site-packages/ultralytics/data/augment.py:1248: RuntimeWarning: invalid value encountered in matmul
  xy = xy @ M.T  # transform


     99/100         0G       1.03       1.83       1.33      1.267        124        640: 100% ━━━━━━━━━━━━ 4/4 0.3it/s 15.1s3.2ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 0.4it/s 5.0s15.0s
                   all         38        302      0.446      0.292        0.3      0.171      0.443      0.261       0.25      0.123

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


/opt/anaconda3/envs/anaconda-python-3-10/lib/python3.10/site-packages/ultralytics/data/augment.py:1248: RuntimeWarning: divide by zero encountered in matmul
  xy = xy @ M.T  # transform
/opt/anaconda3/envs/anaconda-python-3-10/lib/python3.10/site-packages/ultralytics/data/augment.py:1248: RuntimeWarning: overflow encountered in matmul
  xy = xy @ M.T  # transform
/opt/anaconda3/envs/anaconda-python-3-10/lib/python3.10/site-packages/ultralytics/data/augment.py:1248: RuntimeWarning: invalid value encountered in matmul
  xy = xy @ M.T  # transform


    100/100         0G      1.038       1.95      1.306      1.278         89        640: 100% ━━━━━━━━━━━━ 4/4 0.3it/s 15.0s3.2ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 0.4it/s 4.9s15.1s
                   all         38        302      0.437      0.292      0.302      0.173      0.419      0.263      0.244      0.121

100 epochs completed in 0.582 hours.
Optimizer stripped from /Users/artemiy/Library/CloudStorage/GoogleDrive-andrikv@gmail.com/My Drive/University/4 year/cv/runs/segment/yolov11_seg_experiment/weights/last.pt, 6.8MB
Optimizer stripped from /Users/artemiy/Library/CloudStorage/GoogleDrive-andrikv@gmail.com/My Drive/University/4 year/cv/runs/segment/yolov11_seg_experiment/weights/best.pt, 6.8MB

Validating /Users/artemiy/Library/CloudStorage/GoogleDrive-andrikv@gmail.com/My Drive/University/4 year/cv/runs/segment/yolov11_seg_experiment/weights/best.pt...
Ult

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x32d0fa680>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,  

In [19]:
metrics = model.val()

metrics

Ultralytics 8.3.221 🚀 Python-3.10.18 torch-2.9.0 CPU (Apple M4 Max)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 999.8±463.0 MB/s, size: 38.6 KB)
val: Scanning /Users/artemiy/Library/CloudStorage/GoogleDrive-andrikv@gmail.com/My Drive/University/4 year/cv/dataset/valid/labels.cache... 38 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 38/38 155.5Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 0.6it/s 5.4s4.8ss
                   all         38        302      0.585      0.304      0.324      0.208       0.53      0.262      0.261      0.134
                Kolesa         15         20      0.508       0.25      0.312      0.201      0.408        0.2      0.266      0.118
            insiderail         22         41      0.577      0.366      0.359      0.209      0.507      0.317      0.261      0.137
                   lep          8         28      0.528    

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x30d4f7df0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,  

In [21]:
results = model.predict(source="./dataset/test/images", save=True)


image 1/36 /Users/artemiy/Library/CloudStorage/GoogleDrive-andrikv@gmail.com/My Drive/University/4 year/cv/dataset/test/images/190227-03_jpg.rf.b7a4f22bc449516740722fdcaf07cd4b.jpg: 640x640 1 something, 2 trains, 65.7ms
image 2/36 /Users/artemiy/Library/CloudStorage/GoogleDrive-andrikv@gmail.com/My Drive/University/4 year/cv/dataset/test/images/1_jpg.rf.570bc327085ca0d30f6fb49f6abebb69.jpg: 640x640 1 train, 60.0ms
image 3/36 /Users/artemiy/Library/CloudStorage/GoogleDrive-andrikv@gmail.com/My Drive/University/4 year/cv/dataset/test/images/2025-03-18T15_57_37_298_jpg.rf.5b75a9f3ad9534c2ffc4c8431a56e715.jpg: 640x640 1 Kolesa, 1 outsiderail, 1 rail, 58.3ms
image 4/36 /Users/artemiy/Library/CloudStorage/GoogleDrive-andrikv@gmail.com/My Drive/University/4 year/cv/dataset/test/images/2025-03-18T15_57_37_299_jpg.rf.fe3a91219e584804180bf6c80cb6a34d.jpg: 640x640 2 outsiderails, 2 rails, 1 train, 71.4ms
image 5/36 /Users/artemiy/Library/CloudStorage/GoogleDrive-andrikv@gmail.com/My Drive/Univer

In [ ]:
results = model.val(weights="runs/segment/train/exp/weights/best.pt")

print(results.box.map, results.seg.map)

In [ ]:
model.predict(source="datasets/test/images", save=True, imgsz=1024)